### Unzip

In [1]:
import zipfile
import os

def extract_zip_file(zip_file_path, dest_path):
    # Ensure the destination path exists
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(dest_path)
        

# Use the function
zip_file_path = '../archive.zip'  # replace with your zip file path
dest_path = os.getcwd()  # get current working directory
extract_zip_file(zip_file_path, dest_path)

### Data

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)
attr_selected

2023-05-27 21:08:07.452203: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-27 21:08:07.477905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 21:08:07.880495: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


image_id  Attractive  Bangs  Black_Hair  Blond_Hair  Brown_Hair   
0      000001.jpg           1      0           0           0           1  \
1      000002.jpg           0      0           0           0           1   
2      000003.jpg           0      0           0           0           0   
3      000004.jpg           1      0           0           0           0   
4      000005.jpg           1      0           0           0           0   
...           ...         ...    ...         ...         ...         ...   
49995  049996.jpg           0      0           0           1           0   
49996  049997.jpg           1      1           0           0           1   
49997  049998.jpg           0      0           1           0           0   
49998  049999.jpg           1      0           0           0           0   
49999  050000.jpg           1      0           0           0           0   

       Gray_Hair  Male  Oval_Face  Receding_Hairline  Smiling  
0              0     0          0                  0        1  
1              0     0          0                  0        1  
2              0     1          0                  0        0  
3              0     0          0                  0        0  
4              0     0          0                  0        0  
...          ...   ...        ...                ...      ...  
49995          0     0          1                  0        1  
49996          0     0          0                  0        1  
49997          0     1          0                  0        0  
49998          0     0          0                  0        1  
49999          0     0          0                  0        0  

[50000 rows x 11 columns]

In [4]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

In [5]:
features = ['Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']
for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

batch_size = 64
img_height = 128
img_width = 128

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)



Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


### Distribute load

In [ ]:
# Distribute load over all GPU

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def set_gpus(*gpu_indices):
    # Get list of GPUs
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print("Num GPUs Available: ", len(gpus))

    if gpus:
        visible_gpus = [gpus[i] for i in gpu_indices]
        try:
            # Only use specified GPUs and ignore the others
            tf.config.experimental.set_visible_devices(visible_gpus, 'GPU')
            for gpu in visible_gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("Using GPUs: ", [gpu.name for gpu in visible_gpus])
        except RuntimeError as e:
            print(e)
    return visible_gpus

# Call the function with the indices of the GPUs you want to use
visible_gpus = set_gpus(0, 1, 2, 3, 4)  # Use the second, third, fourth, and fifth GPU

# Extract names of the GPUs being used
gpu_names = ['/device:GPU:' + gpu.name.split(':')[-1] for gpu in visible_gpus]

strategy = tf.distribute.MirroredStrategy(devices=gpu_names)

Num GPUs Available:  5
Using GPUs:  ['/physical_device:GPU:0', '/physical_device:GPU:1', '/physical_device:GPU:2', '/physical_device:GPU:3', '/physical_device:GPU:4']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


In [9]:
# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

def _create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3,3), activation = 'relu', input_shape = (input_shape[0], input_shape[1], 3), kernel_regularizer = l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        Conv2D(64, (3,3), activation = 'relu', kernel_regularizer = l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        Conv2D(128, (3,3), activation = 'relu', kernel_regularizer = l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        Conv2D(256, (3,3), activation = 'relu', kernel_regularizer = l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        Conv2D(512, (3,3), activation = 'relu', kernel_regularizer = l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),

        Flatten(),
        Dense(2048, activation = 'relu'),
        BatchNormalization(),
        Dropout(0.25),

        Dense(len(features), activation='sigmoid')

    ])

    return model

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# with strategy.scope():
m1 = _create_model((img_height, img_width))
m1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = m1.fit(train_generator, epochs = 100, validation_data = val_generator, callbacks = [callback])

Epoch 1/100


2023-05-25 23:42:55.857544: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-25 23:42:56.558053: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_12/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


563/563 [==============================] - ETA: 0s - loss: 0.6941 - accuracy: 0.3344

2023-05-25 23:43:48.505959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


563/563 [==============================] - 65s 112ms/step - loss: 0.6941 - accuracy: 0.3344 - val_loss: 0.6497 - val_accuracy: 0.3112
Epoch 2/100
563/563 [==============================] - 62s 110ms/step - loss: 0.4850 - accuracy: 0.3371 - val_loss: 0.4368 - val_accuracy: 0.3448
Epoch 3/100
563/563 [==============================] - 62s 109ms/step - loss: 0.4256 - accuracy: 0.3396 - val_loss: 0.4198 - val_accuracy: 0.3772
Epoch 4/100
563/563 [==============================] - 62s 110ms/step - loss: 0.4098 - accuracy: 0.3382 - val_loss: 0.4027 - val_accuracy: 0.3504
Epoch 5/100
563/563 [==============================] - 63s 112ms/step - loss: 0.4023 - accuracy: 0.3352 - val_loss: 0.4170 - val_accuracy: 0.3504
Epoch 6/100
563/563 [==============================] - 62s 110ms/step - loss: 0.4010 - accuracy: 0.3416 - val_loss: 0.4070 - val_accuracy: 0.3479
Epoch 7/100
563/563 [==============================] - 62s 109ms/step - loss: 0.3984 - accuracy: 0.3414 - val_loss: 0.3910 - val_accurac

In [13]:
loss, accuracy = m1.evaluate(test_generator, steps=len(test_generator), verbose=0)
print(f'Test loss for the model : {loss:.4f}')
print(f'Test accuracy for the model: {accuracy:.4f}')

2023-05-26 04:21:44.079585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test loss for the model : 0.3209
Test accuracy for the model: 0.2836


### Try random search for hyperparameter tuning

In [12]:
from keras import optimizers
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

batch_size = 64
img_height = 128
img_width = 128

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        use_bn = hp.Boolean('use_batch_norm')
        model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                         activation='relu',
                         input_shape=self.input_shape))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=256, step=16),
                         kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_3_filter', min_value=32, max_value=512, step=16),
                         kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_4_filter', min_value=32, max_value=1024, step=16),
                         kernel_size=hp.Choice('conv_4_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Flatten())
        model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=2048, step=16),
                        activation='relu'))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Dense(self.num_classes, activation='sigmoid'))

        model.compile(optimizer=optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        return model

input_shape = (img_height, img_width, 3)  # Assuming your images are RGB, adjust if otherwise
num_classes = len(features)

hypermodel = CNNHyperModel(input_shape, num_classes)

tuner = RandomSearch(hypermodel,
                     objective='val_accuracy',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='random_search',
                     project_name='cnn_hyperparam_tuning')

tuner.search_space_summary()

tuner.search(train_generator, 
             epochs=10, 
             validation_data=val_generator,
             steps_per_epoch=train_generator.samples // batch_size,  # Required due to use of generator
             validation_steps=val_generator.samples // batch_size)  # Required due to use of generator

tuner.results_summary()


Trial 5 Complete [00h 42m 44s]
val_accuracy: 0.4107142786184947

Best val_accuracy So Far: 0.4969865878423055
Total elapsed time: 03h 08m 52s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in random_search/cnn_hyperparam_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
use_batch_norm: False
conv_1_filter: 80
conv_1_kernel: 5
dropout_1: 0.15000000000000002
conv_2_filter: 240
conv_2_kernel: 3
dropout_2: 0.25
conv_3_filter: 464
conv_3_kernel: 5
dropout_3: 0.05
conv_4_filter: 1024
conv_4_kernel: 5
dropout_4: 0.35000000000000003
dense_1_units: 48
learning_rate: 0.009249561789910456
Score: 0.4969865878423055

Trial 0 summary
Hyperparameters:
use_batch_norm: True
conv_1_filter: 96
conv_1_kernel: 5
dropout_1: 0.35000000000000003
conv_2_filter: 208
conv_2_kernel: 3
dropout_2: 0.05
conv_3_filter: 304
conv_3_kernel: 5
dropout_3: 0.0
conv_4_filter: 464
conv_4_kernel: 3
dropout_4: 0.30000000000000004
dense_1_units: 464
le

In [14]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 80)      6080      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 80)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 62, 62, 80)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 240)       173040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 240)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 30, 30, 240)       0

In [15]:
loss, accuracy = best_model.evaluate(test_generator, steps=len(test_generator), verbose=0)
print(f'Test loss for the model : {loss:.4f}')
print(f'Test accuracy for the model: {accuracy:.4f}')

2023-05-26 12:44:07.895210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test loss for the model : 0.5017
Test accuracy for the model: 0.5462


In [16]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss (Custom)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [18]:
best_model.save('custom_model_4_blocks_best.h5')

## 1 feature

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)
attr_selected

2023-05-27 21:20:44.517670: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-27 21:20:44.542321: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 21:20:44.915147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


image_id  Attractive  Bangs  Black_Hair  Blond_Hair  Brown_Hair   
0      000001.jpg           1      0           0           0           1  \
1      000002.jpg           0      0           0           0           1   
2      000003.jpg           0      0           0           0           0   
3      000004.jpg           1      0           0           0           0   
4      000005.jpg           1      0           0           0           0   
...           ...         ...    ...         ...         ...         ...   
49995  049996.jpg           0      0           0           1           0   
49996  049997.jpg           1      1           0           0           1   
49997  049998.jpg           0      0           1           0           0   
49998  049999.jpg           1      0           0           0           0   
49999  050000.jpg           1      0           0           0           0   

       Gray_Hair  Male  Oval_Face  Receding_Hairline  Smiling  
0              0     0          0                  0        1  
1              0     0          0                  0        1  
2              0     1          0                  0        0  
3              0     0          0                  0        0  
4              0     0          0                  0        0  
...          ...   ...        ...                ...      ...  
49995          0     0          1                  0        1  
49996          0     0          0                  0        1  
49997          0     1          0                  0        0  
49998          0     0          0                  0        1  
49999          0     0          0                  0        0  

[50000 rows x 11 columns]

In [11]:
from sklearn.model_selection import train_test_split

features = ['image_id', 'Attractive']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

features = ['Attractive']

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64
img_height = 128
img_width = 128

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)

Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [14]:
from keras import optimizers
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

batch_size = 64
img_height = 128
img_width = 128

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        use_bn = hp.Boolean('use_batch_norm')
        model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                         activation='relu',
                         input_shape=self.input_shape))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=256, step=16),
                         kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_3_filter', min_value=32, max_value=512, step=16),
                         kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_4_filter', min_value=32, max_value=1024, step=16),
                         kernel_size=hp.Choice('conv_4_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Flatten())
        model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=2048, step=16),
                        activation='relu'))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        print(self.num_classes)
        model.add(Dense(self.num_classes, activation='sigmoid'))

        model.compile(optimizer=optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        return model

input_shape = (img_height, img_width, 3)  # Assuming your images are RGB, adjust if otherwise
num_classes = len(features)

hypermodel = CNNHyperModel(input_shape, num_classes)

tuner = RandomSearch(hypermodel,
                     objective='val_accuracy',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='random_search_attractive',
                     project_name='cnn_hyperparam_tuning')

tuner.search_space_summary()

tuner.search(train_generator, 
             epochs=10, 
             validation_data=val_generator,
             steps_per_epoch=train_generator.samples // batch_size,  # Required due to use of generator
             validation_steps=val_generator.samples // batch_size)  # Required due to use of generator

tuner.results_summary()

Trial 5 Complete [01h 06m 15s]
val_accuracy: 0.7869419654210409

Best val_accuracy So Far: 0.7895461320877075
Total elapsed time: 03h 25m 04s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in random_search_attractive/cnn_hyperparam_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
use_batch_norm: True
conv_1_filter: 48
conv_1_kernel: 5
dropout_1: 0.15000000000000002
conv_2_filter: 80
conv_2_kernel: 3
dropout_2: 0.45
conv_3_filter: 208
conv_3_kernel: 3
dropout_3: 0.15000000000000002
conv_4_filter: 160
conv_4_kernel: 5
dropout_4: 0.0
dense_1_units: 1472
learning_rate: 0.004762726870866379
Score: 0.7895461320877075

Trial 4 summary
Hyperparameters:
use_batch_norm: True
conv_1_filter: 80
conv_1_kernel: 3
dropout_1: 0.25
conv_2_filter: 240
conv_2_kernel: 3
dropout_2: 0.25
conv_3_filter: 304
conv_3_kernel: 3
dropout_3: 0.45
conv_4_filter: 832
conv_4_kernel: 5
dropout_4: 0.0
dense_1_units: 1824
learning_rate: 0.00710

In [15]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 48)      3648      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 48)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 62, 62, 48)       192       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 62, 62, 48)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 80)        34640     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 80)      

In [16]:
loss, accuracy = best_model.evaluate(test_generator, steps=len(test_generator), verbose=0)
print(f'Test loss for the model : {loss:.4f}')
print(f'Test accuracy for the model: {accuracy:.4f}')

2023-05-28 00:51:29.002033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test loss for the model : 0.4225
Test accuracy for the model: 0.7978


In [22]:
best_model.save("best_model_attractive.h5")

## Custom model Bangs

In [17]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Bangs']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

features = ['Bangs']

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64
img_height = 128
img_width = 128

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)

Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [19]:
from keras import optimizers
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

batch_size = 64
img_height = 128
img_width = 128

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        use_bn = hp.Boolean('use_batch_norm')
        model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                         activation='relu',
                         input_shape=self.input_shape))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=256, step=16),
                         kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_3_filter', min_value=32, max_value=512, step=16),
                         kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Conv2D(filters=hp.Int('conv_4_filter', min_value=32, max_value=1024, step=16),
                         kernel_size=hp.Choice('conv_4_kernel', values = [3,5]),
                         activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Flatten())
        model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=2048, step=16),
                        activation='relu'))
        if use_bn:
            model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        print(self.num_classes)
        model.add(Dense(self.num_classes, activation='sigmoid'))

        model.compile(optimizer=optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        return model

input_shape = (img_height, img_width, 3)  # Assuming your images are RGB, adjust if otherwise
num_classes = len(features)

hypermodel = CNNHyperModel(input_shape, num_classes)

tuner_bangs = RandomSearch(hypermodel,
                     objective='val_accuracy',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='random_search_bangs',
                     project_name='cnn_hyperparam_tuning')

tuner_bangs.search_space_summary()

tuner_bangs.search(train_generator, 
             epochs=10, 
             validation_data=val_generator,
             steps_per_epoch=train_generator.samples // batch_size,  # Required due to use of generator
             validation_steps=val_generator.samples // batch_size)  # Required due to use of generator

tuner_bangs.results_summary()

Trial 5 Complete [00h 30m 08s]
val_accuracy: 0.9546503027280172

Best val_accuracy So Far: 0.9546503027280172
Total elapsed time: 03h 08m 12s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in random_search_bangs/cnn_hyperparam_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
use_batch_norm: False
conv_1_filter: 48
conv_1_kernel: 5
dropout_1: 0.15000000000000002
conv_2_filter: 240
conv_2_kernel: 5
dropout_2: 0.25
conv_3_filter: 272
conv_3_kernel: 3
dropout_3: 0.05
conv_4_filter: 448
conv_4_kernel: 3
dropout_4: 0.05
dense_1_units: 1792
learning_rate: 0.00015263890159365872
Score: 0.9546503027280172

Trial 0 summary
Hyperparameters:
use_batch_norm: False
conv_1_filter: 112
conv_1_kernel: 3
dropout_1: 0.1
conv_2_filter: 80
conv_2_kernel: 5
dropout_2: 0.0
conv_3_filter: 128
conv_3_kernel: 3
dropout_3: 0.15000000000000002
conv_4_filter: 672
conv_4_kernel: 5
dropout_4: 0.45
dense_1_units: 912
learning_rate: 0.001012

In [20]:
best_model_bangs = tuner_bangs.get_best_models(num_models=1)[0]
best_model_bangs.summary()

1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 48)      3648      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 62, 62, 48)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 58, 240)       288240    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 29, 29, 240)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 29, 29, 240)      

In [21]:
loss, accuracy = best_model_bangs.evaluate(test_generator, steps=len(test_generator), verbose=0)
print(f'Test loss for the model : {loss:.4f}')
print(f'Test accuracy for the model: {accuracy:.4f}')

2023-05-28 04:01:08.738230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test loss for the model : 0.1108
Test accuracy for the model: 0.9536


In [23]:
best_model_bangs.save("best_model_bangs.h5")

## Resnet50

In [ ]:
# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

from keras.applications import ResNet50
from tensorflow.keras.metrics import Precision, Recall


# Define model to transfer learn from 

# Distribute load over all GPU

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def set_gpus(*gpu_indices):
    # Get list of GPUs
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print("Num GPUs Available: ", len(gpus))

    if gpus:
        visible_gpus = [gpus[i] for i in gpu_indices]
        try:
            # Only use specified GPUs and ignore the others
            tf.config.experimental.set_visible_devices(visible_gpus, 'GPU')
            for gpu in visible_gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("Using GPUs: ", [gpu.name for gpu in visible_gpus])
        except RuntimeError as e:
            print(e)
    return visible_gpus

# Call the function with the indices of the GPUs you want to use
visible_gpus = set_gpus(0, 1, 2, 3, 4)  # Use the second, third, fourth, and fifth GPU

# Extract names of the GPUs being used
gpu_names = ['/device:GPU:' + gpu.name.split(':')[-1] for gpu in visible_gpus]

strategy = tf.distribute.MirroredStrategy(devices=gpu_names)

# Create new models with the frozen base models and new top layers
def _create_model(input_shape):
    i = 0
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128,128,3))

    for layer in base_model.layers:
        if i <= 160:
            layer.trainable = False
        i+=1
        
    transfer_model = Sequential()
    transfer_model.add(base_model)
    transfer_model.add(Flatten())
    transfer_model.add(Dense(1024, activation='relu'))
    transfer_model.add(BatchNormalization())
    transfer_model.add(Dropout(0.25))
    transfer_model.add(Dense(len(features), activation='sigmoid'))
    return transfer_model

# Earlystop
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

with strategy.scope():
    m2 = _create_model((img_height, img_width))
    m2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    history = m2.fit(train_generator, epochs = 100, validation_data = val_generator, callbacks = [callback])
    loss, accuracy = m2.evaluate(test_generator, steps = len(test_generator), verbose = 0)

    print("Test Loss:", loss)
    print("Test Acc:", accuracy)

Num GPUs Available:  5
Using GPUs:  ['/physical_device:GPU:0', '/physical_device:GPU:1', '/physical_device:GPU:2', '/physical_device:GPU:3', '/physical_device:GPU:4']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


2023-05-26 06:29:51.367027: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 06:29:51.367213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 06:29:51.367961: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 22 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 22 all-reduces with algorithm = nccl, num_packs = 1


2023-05-26 06:30:19.892230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-26 06:30:22.119685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-26 06:30:24.170143: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-26 06:30:26.396200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-26 06:30:27.961524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-05-26 06:30:30.429690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-26 06:30:33.448408: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fea30760020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-26 06:30:33.448453: I tens

563/563 [==============================] - ETA: 0s - loss: 0.4564 - accuracy: 0.3629

...

Epoch 51/100
563/563 [==============================] - ETA: 0s - loss: 0.3320 - accuracy: 0.3631

2023-05-26 10:33:49.847076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:33:49.847861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:33:50.233811: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:33:50.23

563/563 [==============================] - 301s 535ms/step - loss: 0.3320 - accuracy: 0.3631 - val_loss: 0.4611 - val_accuracy: 0.5221


2023-05-26 10:34:48.769627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:34:48.769994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-26 10:34:48.771448: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_

Test Loss: 0.44487857818603516
Test Acc: 0.5170000195503235


In [14]:
m2.save('resnet50.h5')

In [15]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("ResNet 50 Loss")
plt.ylabel("Loss")
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

## Resnet50: attractive

In [24]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

from keras.applications import ResNet50
from tensorflow.keras.metrics import Precision, Recall

attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Attractive', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair', 'Male', 'Oval_Face', 'Receding_Hairline', 'Smiling']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)
features = ['image_id', 'Attractive']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

features = ['Attractive']

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64
img_height = 128
img_width = 128

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)

Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Create new models with the frozen base models and new top layers
def _create_model(input_shape):
    i = 0
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128,128,3))

    for layer in base_model.layers:
        if i <= 160:
            layer.trainable = False
        i+=1
        
    transfer_model = Sequential()
    transfer_model.add(base_model)
    transfer_model.add(Flatten())
    transfer_model.add(Dense(1024, activation='relu'))
    transfer_model.add(BatchNormalization())
    transfer_model.add(Dropout(0.25))
    transfer_model.add(Dense(len(features), activation='sigmoid'))
    return transfer_model

# Earlystop
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

resnet_attractive = _create_model((img_height, img_width))
resnet_attractive.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = resnet_attractive.fit(train_generator, epochs = 100, validation_data = val_generator, callbacks = [callback])
loss, accuracy = resnet_attractive.evaluate(test_generator, steps = len(test_generator), verbose = 0)

print("Test Loss:", loss)
print("Test Acc:", accuracy)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/100


2023-05-28 04:01:20.930566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-28 04:01:22.922274: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x560b9b6ab500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-28 04:01:22.922290: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2023-05-28 04:01:22.941259: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-28 04:01:23.053971: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directo

563/563 [==============================] - ETA: 0s - loss: 0.6441 - accuracy: 0.6671

2023-05-28 04:02:12.864767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


563/563 [==============================] - 65s 109ms/step - loss: 0.6441 - accuracy: 0.6671 - val_loss: 0.7509 - val_accuracy: 0.6383
Epoch 2/100
563/563 [==============================] - 60s 106ms/step - loss: 0.5836 - accuracy: 0.6966 - val_loss: 0.6960 - val_accuracy: 0.6114
Epoch 3/100
563/563 [==============================] - 60s 106ms/step - loss: 0.5654 - accuracy: 0.7143 - val_loss: 3.4166 - val_accuracy: 0.4843
Epoch 4/100
563/563 [==============================] - 60s 107ms/step - loss: 0.5574 - accuracy: 0.7202 - val_loss: 0.6588 - val_accuracy: 0.6302
Epoch 5/100
563/563 [==============================] - 60s 106ms/step - loss: 0.5504 - accuracy: 0.7210 - val_loss: 0.8688 - val_accuracy: 0.5544
Epoch 6/100
563/563 [==============================] - 60s 107ms/step - loss: 0.5444 - accuracy: 0.7283 - val_loss: 0.9190 - val_accuracy: 0.5728
Epoch 7/100
563/563 [==============================] - 61s 108ms/step - loss: 0.5425 - accuracy: 0.7311 - val_loss: 0.5472 - val_accurac

2023-05-28 04:46:32.644867: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test Loss: 0.6265203952789307
Test Acc: 0.6990000009536743


In [27]:
resnet_attractive.save('resnet_attractive.h5')

## Resnet: bangs

In [28]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2

from keras.applications import ResNet50
from tensorflow.keras.metrics import Precision, Recall


attr = pd.read_csv('list_attr_celeba.csv')
features = ['image_id', 'Bangs']

attr_selected = attr.loc[:, features].iloc[:50000,:]
attr_selected = attr_selected.replace(-1,0)

features = ['Bangs']

train_df, test_df = train_test_split(attr_selected, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

for feature in features:
    train_df[feature] = train_df[feature].astype('float32')
    val_df[feature] = val_df[feature].astype('float32')
    test_df[feature] = test_df[feature].astype('float32')

In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 64
img_height = 128
img_width = 128

train_generator = train_datagen.flow_from_dataframe(train_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

val_generator = train_datagen.flow_from_dataframe(val_df, directory="img_align_celeba/img_align_celeba", 
                                                    x_col="image_id", y_col=features,
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(test_df, directory="img_align_celeba/img_align_celeba", 
                                                  x_col="image_id", y_col=features, 
                                                  target_size=(img_height, img_width), 
                                                  batch_size=batch_size, class_mode='raw', shuffle=False)

Found 36000 validated image filenames.
Found 9000 validated image filenames.
Found 5000 validated image filenames.


In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Create new models with the frozen base models and new top layers
def _create_model(input_shape):
    i = 0
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128,128,3))

    for layer in base_model.layers:
        if i <= 160:
            layer.trainable = False
        i+=1
        
    transfer_model = Sequential()
    transfer_model.add(base_model)
    transfer_model.add(Flatten())
    transfer_model.add(Dense(1024, activation='relu'))
    transfer_model.add(BatchNormalization())
    transfer_model.add(Dropout(0.25))
    transfer_model.add(Dense(len(features), activation='sigmoid'))
    return transfer_model

# Earlystop
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

resnet_bangs = _create_model((img_height, img_width))
resnet_bangs.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = resnet_bangs.fit(train_generator, epochs = 100, validation_data = val_generator, callbacks = [callback])
loss, accuracy = resnet_bangs.evaluate(test_generator, steps = len(test_generator), verbose = 0)

print("Test Loss:", loss)
print("Test Acc:", accuracy)

Epoch 1/100


2023-05-28 04:46:42.598782: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


563/563 [==============================] - ETA: 0s - loss: 0.4156 - accuracy: 0.8386

2023-05-28 04:47:33.527922: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


563/563 [==============================] - 63s 108ms/step - loss: 0.4156 - accuracy: 0.8386 - val_loss: 0.3919 - val_accuracy: 0.8317
Epoch 2/100
563/563 [==============================] - 60s 107ms/step - loss: 0.3236 - accuracy: 0.8683 - val_loss: 0.3742 - val_accuracy: 0.8557
Epoch 3/100
563/563 [==============================] - 60s 107ms/step - loss: 0.2910 - accuracy: 0.8808 - val_loss: 1.8404 - val_accuracy: 0.2930
Epoch 4/100
563/563 [==============================] - 60s 107ms/step - loss: 0.2718 - accuracy: 0.8889 - val_loss: 0.4387 - val_accuracy: 0.8006
Epoch 5/100
563/563 [==============================] - 60s 106ms/step - loss: 0.2615 - accuracy: 0.8939 - val_loss: 1.1608 - val_accuracy: 0.8380
Epoch 6/100
563/563 [==============================] - 60s 106ms/step - loss: 0.2543 - accuracy: 0.8976 - val_loss: 1.5780 - val_accuracy: 0.3221
Epoch 7/100
563/563 [==============================] - 60s 107ms/step - loss: 0.2496 - accuracy: 0.8986 - val_loss: 0.2971 - val_accurac

2023-05-28 05:23:55.841642: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test Loss: 0.2053508758544922
Test Acc: 0.9186000227928162


In [31]:
resnet_bangs.save('resnet_bangs.h5')